---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [14]:
import pandas as pd
import numpy as np

def blight_model():
    import datetime
    from sklearn.svm import LinearSVC
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC
    from sklearn.metrics import roc_curve, auc
    from sklearn.ensemble import RandomForestRegressor
    
    df = pd.read_csv('train.csv', encoding ='latin1', low_memory = False)
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
    df['hearing_date'] = pd.to_datetime(df['hearing_date'])
    df['deadline_length'] = df['hearing_date'] - df['ticket_issued_date']
    df['USA_or_not']= df['country'].apply(lambda x: 1 if x == 'USA' else 0)
    df['MI_or_not']= df['state'].apply(lambda x: 1 if x == 'MI' else 0)
    df.drop(['mailing_address_str_number', 'violation_zip_code', 'violation_description', 'non_us_str_code'], 
            axis = 1, inplace=True)
    df = df[pd.notnull(df['compliance'])]
    
    X = df[['USA_or_not', 'MI_or_not', 'judgment_amount']]
    y = df['compliance']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    clf = RandomForestRegressor(n_estimators = 10, max_depth = 5).fit(X_train, y_train)
#     print('Accuracy of classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
#     print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))

    y_score = clf.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
#     print('auc = {:0.2f}'.format(roc_auc))
    
    df_test = pd.read_csv('test.csv', encoding ='latin1', low_memory = False)
    df_test['ticket_issued_date'] = pd.to_datetime(df_test['ticket_issued_date'])
    df_test['hearing_date'] = pd.to_datetime(df_test['hearing_date'])
    df_test['deadline_length'] = df_test['hearing_date'] - df_test['ticket_issued_date']
    df_test['USA_or_not']= df_test['country'].apply(lambda x: 1 if x == 'USA' else 0)
    df_test['MI_or_not']= df_test['state'].apply(lambda x: 1 if x == 'MI' else 0)
    df_test.drop(['mailing_address_str_number', 'violation_zip_code', 'violation_description', 'non_us_str_code'], 
            axis = 1, inplace=True)
    test = df_test[['USA_or_not', 'MI_or_not', 'judgment_amount']]
#     return df.payment_length.value_counts()
#     return df.iloc[10]
    return pd.DataFrame(clf.predict(test), df_test.ticket_id)



Accuracy of classifier on training set: 0.11
Accuracy of classifier on test set: 0.10
auc = 0.74


0
ticket_id          
284932     0.106175
285362     0.031451
285361     0.106175
285338     0.106175
285346     0.106175
285345     0.106175
285347     0.055350
285342     0.106175
285530     0.035383
284989     0.034902
285344     0.055350
285343     0.031451
285340     0.031451
285341     0.055350
285349     0.106175
285348     0.106175
284991     0.034902
285532     0.034902
285406     0.034902
285001     0.034902
285006     0.031451
285405     0.035383
285337     0.034902
285496     0.055350
285497     0.106175
285378     0.035383
285589     0.034902
285585     0.106175
285501     0.106175
285581     0.031451
...             ...
376367     0.034902
376366     0.036436
376362     0.036436
376363     0.106175
376365     0.034902
376364     0.036436
376228     0.036436
376265     0.036436
376286     0.368130
376320     0.036436
376314     0.036436
376327     0.368130
376385     0.368130
376435     0.034902
376370     0.368130
376434     0.055350
376459     0.106175
376478     0.002014
376473     0.036436
376484     0.018044
376482     0.034902
376480     0.034902
376479     0.034902
376481     0.034902
376483     0.036436
376496     0.031451
376497     0.031451
376499     0.106175
376500     0.106175
369851     0.302699

[61001 rows x 1 columns]

In [ ]:
blight_model()